<center><img src='./Figs/cs-logo.png' width=200></center>



<h6><center></center></h6>

<h1>
<hr style=" border:none; height:3px;">
<center>LAB 3: Harris Detector and Panorama Creation </center>
<hr style=" border:none; height:3px;">
</h1>

The objective of this lab is to develop your own interest point detector using Harris' method (refer to the course).

## Exercise 1: Calculating the Harris Criterion
Here, you are required to write a function or set of functions that return the Harris corner map $ H = det C - \alpha (trace C)^{2}$ for a given image and a given scale (window size). Use $\alpha = 0.04$.


In [1]:
import cv2
import numpy as np

### Step 1
Calculate $I_{x}$ and $I_{y}$ gradient in $x$ and $y$ of a smoothed image using the Sobel operator.

In [2]:
def compute_gradients(img):
    """
    Computes Ix and Iy gradient in x and y of a smoothed image using the Sobel operator
    """
    sobel_x = cv2.Sobel(img, cv2.CV_664F, 1, 0, ksize=3) # cv2.CV_64F: profondeur de l'image de sortie
    sobel_y = cv2.Sobel(img, cv2.CV_664F, 1, 0, ksize=3) # k_size: taille du noyau
    return sobel_x, sobel_y

### Step 2
Calculate $I_{x}^{2}$, $I_{x}^{2}$ and $I_{xy}= I_{x} \times I_{y}$.

In [3]:
def compute_magnitude(img):
    """
    Compute Ix^2, Iy^2 and Ix.Iy
    """
    Ix, Iy = compute_gradients(img)
    return Ix**2, Iy**2, Ix*Iy

### Step 3
Smooth each of the previous images with a Gaussian filter of size $N$.

In [4]:
def gaussian_smooth(img):
    """
    Compute Gaussian Smoothing
    """
    blurred_img = cv2.GaussianBlur(img, (5, 5), 0)
    return blurred_img

### Step 4
At each pixel, calculate the Harris function:$ H = det C - \alpha (trace C)^{2}$ with $\alpha = 0.04

In [5]:
def compute_harris_function(img, alpha=0.04):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray, 2, 3, alpha)
    return dst


Apply to the house image and display the $H$ image.

In [6]:
filename = "Data/house.jpg"
img = cv2.imread(filename)

In [ ]:
cv2.imshow("house.jpg", img)

: 

In [ ]:
dst = compute_harris_function(img)

In [19]:
img[dst>0.01*dst.max()]=[0,0,255]

In [ ]:
cv2.imshow('dst', img)

: 

## Exercise 2: Corner detection

The aim here is to set up functions to detect corners from the corner map constructed in the previous section.

Write a function to binarize the corner map using a thresholding operation.

Write a function to extract local maxima on a $3-$times-3 neighborhood (set to 0 in the binarized image all points whose value is not greater than that of the 8 neighbors). 

Write a function to display the detected points by drawing a white cross at each point on the original image.

Apply it on the house image

## Exercise 3
Several functions are available in OpenCV for calculating points of interest. :
+ [cornerHarris](http://docs.opencv.org/3.2.0/dc/d0d/tutorial_py_features_harris.html)
+ [cornerSubPix](http://docs.opencv.org/3.2.0/dc/d0d/tutorial_py_features_harris.html)
+ [goodFeaturesToTrack](http://docs.opencv.org/3.2.0/d4/d8c/tutorial_py_shi_tomasi.html) which corresponds to Shi Tomasi's approach
+ [SIFT](http://docs.opencv.org/3.2.0/da/df5/tutorial_py_sift_intro.html)

## Exercise 4: Creating a Panorama

The goal of this exercise is to create a panorama from several images, using the main concepts covered in class.

The principle is quite simple and follows the approach seen in class:

+ The first step is to decide which image will be your source image and which will be your destination images (i.e., the images you want to match with your source image).

+ Once the source image is chosen, a technique for creating a panorama is to place this image onto a larger canvas (a larger image where unknown pixels will be set to black). The [**warpAffine**](https://docs.opencv.org/3.4.0/da/d6e/tutorial_py_geometric_transformations.html) function in OpenCV could be used for this.

+ The next step involves detecting and describing a set of key points for the transformed target image and the set of destination images. You can use the [SIFT](https://docs.opencv.org/3.3.0/da/df5/tutorial_py_sift_intro.html) descriptors for this.

+ Then, you need to match the descriptors from the source image with those from the destination image. You can use several [matching](https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_feature2d/py_matcher/py_matcher.html) tools provided by the OpenCV library.

+ Only the top 200 matches will be kept for further processing.

+ From these matches, you need to calculate the homography that transforms the source image into the destination image. Only 4 matches are necessary to calculate this homography, but it is common to use more with the RANSAC approach explained very simply [here](http://eric-yuan.me/ransac/) and available in OpenCV (documentation [here](https://docs.opencv.org/3.4.0/d9/dab/tutorial_homography.html)) as a parameter in the **findHomography** function.

+ Apply the obtained homography to the destination image.

+ Merge the target image and the destination image.

<center><img src='./Figs/panorama.png'></center>


### Import useful files

In [ ]:
%matplotlib inline
import numpy as np
import cv2 
from matplotlib import pyplot as plt

### First step: upload your images and select source and destination images.

You can place your various images in the [`Data`](.\Data) directory.

### Second step: calculating points of interest and their descriptions

For this step, you can try out several of the existing approaches available in the OpenCV library, such as :
+ SIFT (scale-invariant feature transform): see [here](https://docs.opencv.org/master/da/df5/tutorial_py_sift_intro.html)
+ SURF (Speeded-Up Robust Features): very similar to SIFT, but with an implementation that enables rapid calculation of points and descriptors (see [here](https://docs.opencv.org/master/df/dd2/tutorial_py_surf_intro.html)).
+ ORBFASTBRIEF (Oriented FAST and Rotated BRIEF) is a fast binary descriptor based on the combination of the FAST (Features from Accelerated Segment Test) key point detector and the BRIEF (Binary robust independent elementary features) descriptor. It is rotationally invariant and robust to noise. It was developed in the OpenCV labs and is an efficient and free alternative to SIFT. (see [here](https://docs.opencv.org/4.5.1/d1/d89/tutorial_py_orb.html))

### Step 3: Finding matching pairs

The next step is to find candidate matching pairs between two images. Here too, several approaches are possible, such as [`BFMatcher.knnMatch`](https://docs.opencv.org/master/dc/dc3/tutorial_py_matcher.html). This approach measures the distance between each pair of keypoint descriptors and returns for each keypoint its k best matches with the minimum distance.

It is then necessary to apply a ratio filter to keep only the correct matches. Indeed, to obtain a reliable match, the matched key points must be significantly closer than the closest incorrect match.

### Step 4: Calculating homography 

Once we've matched at least four pairs of key points, we can transform one image relative to the other. This operation is called image warping. Two images of the same plane surface in space are linked by a homography. Homographies are geometric transformations that have 8 free parameters and are represented by a 3x3 matrix. They represent any distortion applied to an image as a whole (as opposed to local distortions). Consequently, to obtain the transformed detected image, we need to calculate the homography matrix and apply it to the detected image.

The RANSAC algorithm can be used to detect outliers and eliminate them before determining the final homography. It is directly integrated into OpenCV's [`findHomography`](https://docs.opencv.org/master/d1/de0/tutorial_py_feature_homography.html) method. 

### Fifth step: image warping

Apply a warp transformation using the homography matrix obtained

Apply your chain to the various imahes collected. What do you recommend? What kind of improvements would you suggest?

To go deeper:
+ https://www.pyimagesearch.com/2018/12/17/image-stitching-with-opencv-and-python/